# Image Retrieval for Visual Geolocalization: Extensions and Experiments

## Libraries

In [5]:
# Import libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from itertools import product
from pytorch_metric_learning import miners, losses
from pytorch_metric_learning.distances import CosineSimilarity, DotProductSimilarity
from torch.optim import SGD, Adam, AdamW, ASGD, RMSprop
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from tqdm import tqdm

In [10]:
# Import Modules
from datasets.Train import TrainDataset, transform
from datasets.Eval import EvalDataset
from visualization.Visualization import print_sample_dataset
# mettere print_preds in eval
from models.Aggregators import Avg_ResNet
from models.Aggregators import GeM_ResNet
from models.Training_loop import training_loop
from models.Evaluation_loop import evaluation_loop

ImportError: cannot import name 'print_sample_dataset' from 'visualization' (unknown location)

## Initializations

In [3]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Set all manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)

## Load Datasets


In [ ]:
# Training loading
root_dir_train = '/kaggle/input/gsv-xs/gsv_xs/train'
dataset_train = TrainDataset(root_dir=root_dir_train, transform=transform)
dataloader_train = data.DataLoader(dataset_train, batch_size=64, shuffle=True)

In [ ]:
# Validation loading
root_dir_eval = '/kaggle/input/sf-xs/sf_xs'
dataset_val = EvalDataset(root_dir=root_dir_eval, type_of_set= 'val', transform=transform)
dataloader_val = data.DataLoader(dataset_val, batch_size=64, shuffle=False)

In [ ]:
# Test loading

# SF-XS
root_dir_eval = '/kaggle/input/sf-xs/sf_xs'
dataset_test = EvalDataset(root_dir=root_dir_eval, type_of_set= 'test', transform=transform)
dataloader_test = data.DataLoader(dataset_test, batch_size=64, shuffle=False)

# Tokyo-xs
root_dir_tokyo = '/kaggle/input/tokyo-xs/tokyo_xs'
dataset_tokyo = EvalDataset(root_dir=root_dir_tokyo, type_of_set= 'test', transform=transform)
dataloader_tokyo = data.DataLoader(dataset_tokyo, batch_size=64, shuffle=False)

## First Visualizations

In [ ]:
print_sample_dataset('train', 1, 3, dataloader_train, dataloader_tokyo)

In [ ]:
print_sample_dataset('test', 1, 3, dataloader_train, dataloader_tokyo)

## Network

#### AVG

In [ ]:
# if torch.cuda.device_count() > 1:  # multiple GPU in parallel
#     print("Let's use", torch.cuda.device_count(), "GPUs")
#     model_avg = nn.DataParallel(Avg_ResNet())
#     model_avg = model_avg.cuda()
# else:  # single GPU
#     model_avg = Avg_ResNet().cuda()

# # Print the model architecture
# print(model_avg)

# # Save the model's initial state dictionary
# torch.save(model_avg.state_dict(), '/kaggle/working/initial_weights.pth')

#### GEM

In [ ]:
# Initialize the network
if torch.cuda.device_count() > 1:  # multiple GPU in parallel
    print("Let's use", torch.cuda.device_count(), "GPUs")
    model_gem = nn.DataParallel(GeM_ResNet())
    model_gem = model_gem.cuda()
else:  # single GPU
    model_gem = GeM_ResNet().cuda()

# Print the model architecture
print(model_gem)

# Save the model's initial state dictionary
torch.save(model_gem.state_dict(), '/kaggle/working/initial_weights.pth')

In [ ]:
# Loading saved weights
model_avg.load_state_dict(torch.load('/kaggle/input/pesiiii/model_weights.pth'))  

## Training session

#### Criterion and Optimizer

In [ ]:
criterion = losses.ContrastiveLoss(pos_margin=0, neg_margin=1)
optimizer = SGD(model_gem.parameters(), lr=0.0001, weight_decay=0.0001, momentum=0.9)

#### Miner

In [ ]:
miner = None

#### Save model's weights

In [ ]:
torch.save(model_avg.state_dict(), 'model_weights.pth')

#### Training loop

In [ ]:
print('\033[1;31mRESULTS ON TRAINING\033[0m')
for epoch in tqdm(range(1,11)):
    training_loss = training_loop(epoch, model_gem, dataloader_train, criterion, optimizer, miner = miner, pre_miner = 'Proxy')                   
    if sched_name == 'CosineAnnealingLR':            
        scheduler.step()

## Validate session 
DA METTERE QUELLE CHE HANNO GIRATO

In [ ]:
# # FIRST GRID SEARCH 

# # Parametri per la grid search
# optimizers = ['SGD', 'Adam', 'ASGD', 'AdamW', 'RMSprop']  
# momentums = [0, 0.95]  # [0.0, 0.8, 0.9, 0.95]  # Tuning sul parametro momentum per SGD

# # Risultati della grid search
# results = []

# # Loop di grid search
# for opt_name, momentum in product(optimizers, momentums):
#     print(f'Running with LR={lr}, WD={wd}, Optimizer={opt_name}, Scheduler={sched_name}, Momentum={momentum}')
    
#     # Scegli l'ottimizzatore
#     if opt_name == 'SGD':
#         optimizer = optim.SGD(model_avg.parameters(), momentum=momentum)
#     elif opt_name == 'Adam':
#         optimizer = optim.Adam(model_avg.parameters())
#     elif opt_name == 'AdamW':
#         optimizer = optim.AdamW(model_avg.parameters())
#     elif opt_name == 'ASGD':
#         optimizer = optim.ASGD(model_avg.parameters())
#     elif opt_name == 'RMSprop':
#         optimizer = optim.RMSprop(model_avg.parameters(), momentum=momentum)
      
#     # Loop di addestramento
#     num_epochs = 10
#     print('\033[1;31mRESULTS ON TRAINING\033[0m')
#     for epoch in tqdm(range(num_epochs)):
#         training_loss = training_loop(epoch, model_avg, dataloader_train, criterion, optimizer)
#         # validation_loss = evaluation_loop(dataset_val, model_avg, dataloader_val, k_values, True,3)
   
#     # Salva i risultati
#     results.append({
#         'optimizer': opt_name,
#         'momentum': momentum,
#         'final_loss': training_loss
#     })

#     evaluation_loop(dataset_val, model_avg, dataloader_val, k_values, False)
#     model_avg.reset_parameters()  # vedere se worka
    
# # Stampa i risultati finali
# for result in results:
#     print(result)

In [ ]:
k_values= [1,5]
# Parametri per la grid search
learning_rates = [1e-4]  # [1e-4, 1e-5]  # [1e-5, 1e-4, 1e-3, 1e-2]  
weight_decays = [1e-3]  
optimizers = ['Adam']  #, 'AdamW']  # best optimizers
# momentums = [0, 0.95]  # [0.0, 0.8, 0.9, 0.95]  
schedulers = ['CosineAnnealingLR']
miner_names = ['AngularMiner']

# Risultati della grid search
results = []

# Loop di grid search
for lr, wd, opt_name, sched_name, miner_name in product(learning_rates, weight_decays, optimizers, schedulers, miner_names):
    print(f'Running with LR={lr}, WD={wd}, Optimizer={opt_name}, Scheduler={sched_name}')
    
    # Scegli l'ottimizzatore
    if opt_name == 'Adam':
        optimizer = optim.Adam(model_gem.parameters(), lr=lr, weight_decay=wd)

    # Scegli lo scheduler
    if sched_name == 'CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=10, verbose=True)
    
    if miner_name == 'MultiSimilarityMiner':
        miner = miners.MultiSimilarityMiner(epsilon=0.1)
    elif miner_name == 'TripletMarginMiner':
        miner = miners.TripletMarginMiner(margin=0.2, type_of_triplets="all")
    elif miner_name == 'BatchHardMiner':
        miner = miners.BatchHardMiner()
    elif miner_name == 'AngularMiner':
        miner = miners.AngularMiner(angle=20)
    
    # Loop di addestramento
    num_epochs = 10
    print('\033[1;31mRESULTS ON TRAINING\033[0m')
    for epoch in tqdm(range(1,13)):
        training_loss = training_loop(epoch, model_gem, dataloader_train, criterion, optimizer, miner = miner, pre_miner = 'Proxy')                   
        if sched_name == 'CosineAnnealingLR':            
            scheduler.step()
    
    recalls = evaluation_loop(dataset_val, model_gem, dataloader_val, k_values, False)
    model_gem.load_state_dict(torch.load('/kaggle/input/initial-weights-gem-parallel/initial_weights_gem_parallel.pth'))  

    # Salva i risultati
    results.append({
        'optimizer': opt_name,
        'recall@1': recalls[0],
        'recall@5': recalls[1]
    })
    
# Stampa i risultati finali
for result in results:
    print(result)

In [ ]:
# # GRID SEARCH FOR LOSSES
# k_values = [1, 5]
# num_classes = dataset_train.__len__()

# # Parametri per la grid search
# ArcFaceLoss = losses.ArcFaceLoss(num_classes, embedding_size=256, margin=28.6, scale=64)
# ContrastiveLoss = losses.ContrastiveLoss(pos_margin=0, neg_margin=1)
# CosFaceLoss = losses.CosFaceLoss(num_classes, embedding_size=256, margin=0.35, scale=64)
# MultiSimilarityLoss = losses.MultiSimilarityLoss(alpha=1.0, beta=50, base=0.0, distance=DotProductSimilarity())
# type_losses = [ArcFaceLoss, ContrastiveLoss, CosFaceLoss, MultiSimilarityLoss]
# # Proxy in dolce attesa delle idee del maestro GAZA
# # N-pair Loss --> buona, ma richiede batch molto grande, quindi con i  nostri strimenti infatttibile --> SCRIVERE NELLE ESTENSIONI

# # Risultati della grid search
# results = []

# # Loop di grid search
# for loss in type_losses:
#     print(f'Loss: {loss}')
    
#     # Loop di addestramento
#     num_epochs = 3
#     print('\033[1;31mRESULTS ON TRAINING\033[0m')
#     for epoch in tqdm(range(num_epochs)):
#         training_loss = training_loop(epoch, model_avg, dataloader_train, loss, optimizer)
#         # validation_loss = evaluation_loop(dataset_val, model_avg, dataloader_val, k_values, True,3)

#     # Salva i risultati
#     results.append({'loss': loss, 'final_loss': training_loss})
#     evaluation_loop(dataset_val, model_avg, dataloader_val, k_values, False)
#     model_avg.load_state_dict(torch.load('/kaggle/working/initial_weights.pth'))
#     print('---------------------------------------------------------------')

# # Stampa i risultati finali
# for result in results:
#     print(result)

## Test session

In [ ]:
k_values = [1, 5]

print('\033[1;32mRESULTS ON SF-XS VAL\033[0m')
evaluation_loop(dataset_val, model_gem, dataloader_val, k_values, False)

print('\033[1;33mRESULTS ON SF-XS TEST\033[0m')
evaluation_loop(dataset_test, model_gem, dataloader_test, k_values, False)

print('\033[1;36mRESULTS ON TOKYO TEST\033[0m')
evaluation_loop(dataset_tokyo, model_gem, dataloader_tokyo, k_values, False)